# First, You have to know linking data

In [18]:
import numpy as np
import pandas as pd

Get dfs_pm for pm data and dfs_info for pm station informations

In [19]:
dfs_pm = {}
dfs_info = {}

for i in range(2011, 2025):
    dfs_pm[i] = pd.read_excel(f"../../data/pm2-5/Raw_Data/PM2.5/PM2.5({i}).xlsx", sheet_name="PM2.5")
    if i == 2023:
        dfs_pm[i] = dfs_pm[i].iloc[:-3]
    else:
        dfs_pm[i] = dfs_pm[i].iloc[:-1]
    dfs_pm[i] = dfs_pm[i].dropna(how='all', subset=dfs_pm[i].columns[1:])
    dfs_pm[i].columns = dfs_pm[i].columns.str.strip()
    sheet_names = pd.ExcelFile(f"../../data/pm2-5/Raw_Data/PM2.5/PM2.5({i}).xlsx").sheet_names
    if "รายละเอียด" in sheet_names:
        dfs_info[i] = pd.read_excel(
            f"../../data/pm2-5/Raw_Data/PM2.5/PM2.5({i}).xlsx", sheet_name="รายละเอียด", skiprows=2, usecols=lambda x: not x.startswith('Unnamed') and x!= "ลำดับ")
    elif "พารามิเตอร์_สถานี" in sheet_names:
        dfs_info[i] = pd.read_excel(f"../../data/pm2-5/Raw_Data/PM2.5/PM2.5({i}).xlsx", sheet_name="พารามิเตอร์_สถานี", skiprows=2, usecols=lambda x: not x.startswith('Unnamed') and x!= "ลำดับ")
    elif "รายละเอียดจุดตรวจวัด" in sheet_names:
        dfs_info[i] = pd.read_excel(f"../../data/pm2-5/Raw_Data/PM2.5/PM2.5({i}).xlsx", sheet_name="รายละเอียดจุดตรวจวัด", skiprows=2, usecols=lambda x: not x.startswith('Unnamed') and x!= "ลำดับ")
    else:
        dfs_info[i] = None
        print(f"No matching sheet found for year {i}")
    
    dfs_info[i] = dfs_info[i].drop(index=0)
    print(f"Year {i} loaded")

Year 2011 loaded
Year 2012 loaded
Year 2013 loaded
Year 2014 loaded
Year 2015 loaded
Year 2016 loaded
Year 2017 loaded
Year 2018 loaded
Year 2019 loaded
Year 2020 loaded
Year 2021 loaded
Year 2022 loaded
Year 2023 loaded
Year 2024 loaded


We have data from 2011 to 2025. oddly, for dfs_pm, I found the problem on year 2023 which last three row didn't be used and other years didn't use the last row. Lastly, I have to make sure that no data with all NaN except the first columns, so I dropped them and strip all column name (yes I also found it unique if not doing it).

about dfs_info, each files have different sheet_names that necessary to use condition to check.

next, concat all years into one.

In [20]:
df_pm = pd.concat(dfs_pm.values(), ignore_index=True).sort_values(by='Date', ascending=False).reset_index(drop=True)

dont forget to check if columns duplicate (that's mean u didn't strips)

In [21]:
list(df_pm.columns)

['Date',
 '54T',
 '36T',
 '30T',
 '24T',
 '44T',
 '27T',
 '40T',
 '61T',
 '26T',
 '46T',
 '59T',
 '32T',
 '05T',
 '52T',
 '08T',
 '35T',
 '75T',
 '76T',
 '77T',
 '50T',
 '53T',
 '19T',
 '62T',
 '63T',
 '79T',
 '80T',
 '10T',
 '11T',
 '03T',
 '14T',
 '17T',
 '18T',
 '20T',
 '21T',
 '25T',
 '29T',
 '31T',
 '33T',
 '34T',
 '37T',
 '38T',
 '39T',
 '41T',
 '42T',
 '57T',
 '58T',
 '67T',
 '68T',
 '69T',
 '70T',
 '71T',
 '72T',
 '73T',
 '02T',
 '12T',
 '13T',
 '16T',
 '22T',
 '28T',
 '43T',
 '47T',
 '60T',
 '70T*',
 '70T**',
 '74T',
 '78T',
 '81T',
 '82T',
 '83T',
 '84T',
 '85T',
 '72T(เก่า)',
 '72T(ใหม่)',
 '88T',
 '90T',
 '91T',
 '87T',
 '86T',
 '92T',
 '89T',
 '93T',
 '99T',
 '103T',
 '101T',
 '102T',
 '100T',
 '94T',
 '95T',
 '96T',
 '97T',
 '98T',
 '104T',
 '106T',
 '107T',
 '108T',
 '109T',
 '110T',
 '111T',
 '112T',
 '105T']

well, now concat to df_info. not all sheet that columns name ID, so I renamed it and then concat.

In [22]:
for i in range(2011, 2025):
    if dfs_info[i] is not None:
        if "ID" in dfs_info[i].columns:
            dfs_info[i].rename(columns={"ID": "รหัสสถานี"}, inplace=True)
        else:
            print(f"No ID column found in year {i}")
    else:
        print(f"No info data for year {i}")

df_info = pd.concat(dfs_info.values(), ignore_index=True)

No ID column found in year 2018
No ID column found in year 2019
No ID column found in year 2020
No ID column found in year 2021
No ID column found in year 2022
No ID column found in year 2023
No ID column found in year 2024


Then I found one station named differently. I renamed it.

In [23]:
for i in range(2011, 2025):
    dfs_info[i].loc[dfs_info[i]['รหัสสถานี'] == 70, 'รหัสสถานี'] = "70T"
    dfs_info[i]['รหัสสถานี'] = dfs_info[i]['รหัสสถานี'].str.strip()

After that, drop duplicates.

In [24]:
df_info.drop_duplicates(subset=["รหัสสถานี"], inplace=True)

pd.options.display.max_rows = None
pd.options.display.max_columns = None

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
# df_info

I have to make sure about the amount of the station so...

In [25]:
len(df_info["รหัสสถานี"].unique())
df_pm.columns[df_pm.columns.duplicated()]

Index([], dtype='object')

In [26]:
df_pm.shape

(4979, 101)

now, station number of both dfs are matched

In [27]:
df_info['รหัสสถานี'][~df_info['รหัสสถานี'].isin(df_pm.columns)]

321            70
323    72T (เก่า)
324    72T (ใหม่)
Name: รหัสสถานี, dtype: object

In [28]:
df_pm.columns[~df_pm.columns.isin(df_info['รหัสสถานี'])]

Index(['Date', '72T(เก่า)', '72T(ใหม่)'], dtype='object')

yeah, U did see the problem above so I did this.

In [29]:
df_info.loc[df_info['รหัสสถานี'] == 70, 'รหัสสถานี'] = "70T"
df_info['รหัสสถานี'] = df_info['รหัสสถานี'].str.replace(" ", "")


In [30]:
df_info['รหัสสถานี'][~df_info['รหัสสถานี'].isin(df_pm.columns)]

Series([], Name: รหัสสถานี, dtype: object)

In [31]:
df_pm.columns[~df_pm.columns.isin(df_info['รหัสสถานี'])]

Index(['Date'], dtype='object')

now, it's ok

lets rename the row

before that, recheck again.

In [33]:
for x in df_pm.columns[1:]:
    print(f"processing: {x}")
    if x not in df_info["รหัสสถานี"].values:
        print(f"{x} is not in df_info")

processing: 54T
processing: 36T
processing: 30T
processing: 24T
processing: 44T
processing: 27T
processing: 40T
processing: 61T
processing: 26T
processing: 46T
processing: 59T
processing: 32T
processing: 05T
processing: 52T
processing: 08T
processing: 35T
processing: 75T
processing: 76T
processing: 77T
processing: 50T
processing: 53T
processing: 19T
processing: 62T
processing: 63T
processing: 79T
processing: 80T
processing: 10T
processing: 11T
processing: 03T
processing: 14T
processing: 17T
processing: 18T
processing: 20T
processing: 21T
processing: 25T
processing: 29T
processing: 31T
processing: 33T
processing: 34T
processing: 37T
processing: 38T
processing: 39T
processing: 41T
processing: 42T
processing: 57T
processing: 58T
processing: 67T
processing: 68T
processing: 69T
processing: 70T
processing: 71T
processing: 72T
processing: 73T
processing: 02T
processing: 12T
processing: 13T
processing: 16T
processing: 22T
processing: 28T
processing: 43T
processing: 47T
processing: 60T
processi

In [35]:
for x in df_info["รหัสสถานี"].values:
    print(f"processing: {x}")
    if x not in df_pm.columns:
        print(f"{x} is not in df_pm")

processing: 54T
processing: 30T
processing: 36T
processing: 24T
processing: 44T
processing: 27T
processing: 40T
processing: 61T
processing: 26T
processing: 46T
processing: 59T
processing: 32T
processing: 05T
processing: 52T
processing: 08T
processing: 35T
processing: 75T
processing: 76T
processing: 77T
processing: 50T
processing: 53T
processing: 19T
processing: 62T
processing: 63T
processing: 79T
processing: 80T
processing: 10T
processing: 11T
processing: 03T
processing: 14T
processing: 17T
processing: 18T
processing: 20T
processing: 21T
processing: 25T
processing: 29T
processing: 31T
processing: 33T
processing: 34T
processing: 37T
processing: 38T
processing: 39T
processing: 41T
processing: 42T
processing: 57T
processing: 58T
processing: 67T
processing: 68T
processing: 69T
processing: 70T
processing: 71T
processing: 72T
processing: 73T
processing: 02T
processing: 12T
processing: 13T
processing: 16T
processing: 22T
processing: 28T
processing: 43T
processing: 47T
processing: 60T
processi

In [36]:
df_pm.rename(columns={df_info["รหัสสถานี"].values[i]: f"{df_info['รหัสสถานี'].values[i]} | {df_info['ชื่อสถานี'].values[i]}" for i in range(len(df_info))}, inplace=True)

In [37]:
df_pm

,Date,54T | แขวงดินแดง เขตดินแดง กทม.,36T | ต.ศรีภูมิ อ.เมือง จ.เชียงใหม่,30T | ต.ท่าประดู่ อ.เมือง จ.ระยอง,24T | ต.หน้าพระลาน อ.เฉลิมพระเกียรติ จ.สระบุรี,44T | ต.หาดใหญ่ อ.หาดใหญ่ จ.สงขลา,27T | ต.มหาชัย อ.เมือง จ.สมุทรสาคร,40T | ต.แม่เมาะ อ.แม่เมาะ จ.ลำปาง,61T | แขวงพลับพลา เขตวังทองหลาง กทม.,26T | ต.หน้าเมือง อ.เมือง จ.ราชบุรี,...,98T | ต.อุทัยใหม่ อ.เมืองอุทัยธานี จ.อุทัยธานี,104T | ต.บางมัญ อ.เมือง จ.สิงห์บุรี,106T | ต.บึงกาฬ อ.เมือง จ.บึงกาฬ,107T | ต.กาฬสินธุ์ อ.เมือง จ.กาฬสินธุ์,108T | ต.ในเมือง อ.เมือง จ.ชัยภูมิ,109T | ต.ในเมือง อ.เมือง จ.ยโสธร,110T | ต.บุ่ง อ.เมือง จ.อำนาจเจริญ,111T | ต.นอกเมือง อ.เมือง จ.สุรินทร์,112T | ต.หนองครก อ.เมือง จ.ศรีสะเกษ,105T | ต.บ้านใหญ่ อ.เมือง จ.นครนายก
0,2024-12-31 00:00:00,34.9,21.5,25.6,37.1,15.6,38.9,22.0,29.5,41.2,...,35.2,32.0,20.4,33.9,23.3,33.9,28.6,19.5,20.0,23.8
1,2024-12-30 00:00:00,33.4,23.1,23.4,29.6,14.0,49.2,20.9,27.1,35.1,...,37.9,38.6,24.9,32.2,27.1,38.9,31.8,25.2,30.2,19.6
2,2024-12-29 00:00:00,23.3,20.1,12.3,20.8,10.1,30.9,20.2,20.1,27.1,...,24.9,25.1,15.1,32.9,17.9,35.3,29.5,23.2,22.5,12.0
3,2024-12-28 00:00:00,27.7,17.4,13.4,25.3,9.7,43.8,23.7,22.6,NaN,...,37.8,24.4,10.1,20.4,19.3,22.7,17.4,9.7,9.8,14.4
4,2024-12-27 00:00:00,44.5,23.6,24.4,35.6,12.0,63.6,27.3,34.6,47.1,...,51.2,30.3,10.7,23.5,20.7,22.9,16.9,12.4,10.1,36.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4974,2011-05-20 00:00:00,43,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4975,2011-05-19 00:00:00,49,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4976,2011-05-18 00:00:00,45,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4977,2011-05-17 00:00:00,45,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
list = df_pm.columns

for x in list:
    print(x)

Date
54T |  แขวงดินแดง เขตดินแดง กทม.
36T |  ต.ศรีภูมิ อ.เมือง จ.เชียงใหม่
30T |  ต.ท่าประดู่ อ.เมือง จ.ระยอง
24T |  ต.หน้าพระลาน อ.เฉลิมพระเกียรติ จ.สระบุรี 
44T |  ต.หาดใหญ่ อ.หาดใหญ่ จ.สงขลา
27T |  ต.มหาชัย อ.เมือง จ.สมุทรสาคร
40T | ต.แม่เมาะ อ.แม่เมาะ จ.ลำปาง
61T |  แขวงพลับพลา เขตวังทองหลาง กทม.
26T |  ต.หน้าเมือง อ.เมือง จ.ราชบุรี
46T |  ต.ในเมือง อ.เมือง จ.ขอนแก่น
59T |  แขวงพญาไท เขตพญาไท กทม.
32T |  ต.ทุ่งสุขลา อ.ศรีราชา จ.ชลบุรี
05T |  แขวงบางนา เขตบางนา กทม.
52T | แขวงบางยี่เรือ เขตธนบุรี กทม.
08T |  ต.ทรงคนอง  อ.พระประแดง  จ.สมุทรปราการ
35T |  ต.ช้างเผือก อ.เมือง จ.เชียงใหม่
75T | ต.ห้วยโก๋น อ.เฉลิมพระเกียรติ จ.น่าน
76T |  ต.แม่ปะ อ.แม่สอด จ.ตาก
77T |  ต.ท่าตูม อ.ศรีมหาโพธิ จ.ปราจีนบุรี
50T |  แขวงปทุมวัน เขตปทุมวัน กทม.
53T |  แขวงวังทองหลาง เขตวังทองหลาง กทม.
19T |  ต.บางเสาธง อ.บางเสาธง จ.สมุทรปราการ
62T |  ต.บางนาค อ.เมือง จ.นราธิวาส
63T |  ต.สะเตง อ.เมือง จ.ยะลา
79T |  ต.ปากแพรก อ.เมือง จ.กาญจนบุรี
80T |  ต.พิมาน อ.เมือง. จ.สตูล
10T |  แขวงคลองจั่น เขตบางกะปิ กทม.
11T 

### great job!

Next part is... wtf did i do

Oh! I filter the data.

Get the province name from the data

In [39]:
print(df_info["ชื่อสถานี"])

df_info["ชื่อสถานี"] = df_info["ชื่อสถานี"].str.strip()

0                       แขวงดินแดง เขตดินแดง กทม.
1                     ต.ท่าประดู่ อ.เมือง จ.ระยอง
2                   ต.ศรีภูมิ อ.เมือง จ.เชียงใหม่
4       ต.หน้าพระลาน อ.เฉลิมพระเกียรติ จ.สระบุรี 
7                     ต.หาดใหญ่ อ.หาดใหญ่ จ.สงขลา
                          ...                    
479                     ต.ในเมือง อ.เมือง จ.ยโสธร
480                   ต.บุ่ง อ.เมือง จ.อำนาจเจริญ
481                 ต.นอกเมือง อ.เมือง จ.สุรินทร์
482                  ต.หนองครก อ.เมือง จ.ศรีสะเกษ
496                  ต.บ้านใหญ่ อ.เมือง จ.นครนายก
Name: ชื่อสถานี, Length: 101, dtype: object


In [40]:
df_info["จังหวัด"] = df_info["ชื่อสถานี"].str.split(" ").str[-1]

In [41]:
df_info["จังหวัด"].unique()

array(['กทม.', 'จ.ระยอง', 'จ.เชียงใหม่', 'จ.สระบุรี', 'จ.สงขลา',
       'จ.สมุทรสาคร', 'จ.ลำปาง', 'จ.ราชบุรี', 'จ.ขอนแก่น', 'จ.ชลบุรี',
       'จ.สมุทรปราการ', 'จ.น่าน', 'จ.ตาก', 'จ.ปราจีนบุรี', 'จ.นราธิวาส',
       'จ.ยะลา', 'จ.กาญจนบุรี', 'จ.สตูล', 'จ.ปทุมธานี',
       'จ.พระนครศรีอยุธยา', 'จ.นครสวรรค์', 'จ.สุราษฏร์ธานี', 'จ.เชียงราย',
       'จ.แม่ฮ่องสอน', 'จ.ลำพูน', 'แพร่', 'พะเยา', 'จ.สระแก้ว', 'จ.เลย',
       'จ.นนทบุรี', 'จ.ภูเก็ต', 'จ.นครราชสีมา', 'จ.ฉะเชิงเทรา', 'จ.พะเยา',
       'จ.นครปฐม', 'จ.หนองคาย', 'จ.อุบลราชธานี', 'จ.สมุทรสงคราม',
       'จ.สุพรรณบุรี', 'จ.พิษณุโลก', 'จ.ตราด', 'จ.นครพนม',
       'จ.นครศรีธรรมราช', 'จ.สกลนคร', 'จ.อุดรธานี', 'จ.อุตรดิตถ์',
       'จ.ตรัง', 'จ.ลพบุรี', 'จ.ประจวบคีรีขันธ์', 'จ.บุรีรัมย์',
       'จ.มุกดาหาร', 'จ.กำแพงเพชร', 'จ.พิจิตร', 'จ.สุโขทัย',
       'จ.เพชรบูรณ์', 'จ.อุทัยธานี', 'จ.สิงห์บุรี', 'จ.บึงกาฬ',
       'จ.กาฬสินธุ์', 'จ.ชัยภูมิ', 'จ.ยโสธร', 'จ.อำนาจเจริญ',
       'จ.สุรินทร์', 'จ.ศรีสะเกษ', 'จ.นครนายก'], dtype=object)

In [44]:
thai_to_eng_map = {
    "กทม.": ("Bangkok Metropolis", 3),
    "สมุทรปราการ": ("Samut Prakan", 57),
    "นนทบุรี": ("Nonthaburi", 36),
    "สมุทรสาคร": ("Samut Sakhon", 58),
    "ปทุมธานี": ("Pathum Thani", 37),
    "นครปฐม": ("Nakhon Pathom", 27),
    "พระนครศรีอยุธยา": ("Phra Nakhon Si Ayutthaya", 46),
    "ราชบุรี": ("Ratchaburi", 52),
    "กาญจนบุรี": ("Kanchanaburi", 16),
    "สมุทรสงคราม": ("Samut Songkhram", 59),
    "สุพรรณบุรี": ("Suphan Buri", 66),
    "ลพบุรี": ("Lop Buri", 22),
    "ประจวบคีรีขันธ์": ("Prachuap Khiri Khan", 50),
    "สิงห์บุรี": ("Sing Buri", 63),
    "ขอนแก่น": ("Khon Kaen", 17),
    "นครราชสีมา": ("Nakhon Ratchasima", 29),
    "เลย": ("Loei", 21),
    "หนองคาย": ("Nong Khai", 35),
    "อุบลราชธานี": ("Ubon Ratchathani", 72),
    "นครพนม": ("Nakhon Phanom", 28),
    "สกลนคร": ("Sakon Nakhon", 56),
    "อุดรธานี": ("Udon Thani", 73),
    "บุรีรัมย์": ("Buri Ram", 5),
    "มุกดาหาร": ("Mukdahan", 25),
    "บึงกาฬ": ("Bueng Kan", 4),
    "กาฬสินธุ์": ("Kalasin", 14),
    "ชัยภูมิ": ("Chaiyaphum", 8),
    "ยโสธร": ("Yasothon", 77),
    "อำนาจเจริญ": ("Amnat Charoen", 1),
    "สุรินทร์": ("Surin", 68),
    "ศรีสะเกษ": ("Si Sa Ket", 62),
    "ระยอง": ("Rayong", 53),
    "ชลบุรี": ("Chon Buri", 12),
    "ฉะเชิงเทรา": ("Chachoengsao", 6),
    "ปราจีนบุรี": ("Prachin Buri", 49),
    "ตราด": ("Trat", 71),
    "จันทบุรี": ("Chanthaburi", 9),
    "นครนายก": ("Nakhon Nayok", 26),
    "เชียงใหม่": ("Chiang Mai", 10),
    "ลำปาง": ("Lampang", 19),
    "นครสวรรค์": ("Nakhon Sawan", 30),
    "เชียงราย": ("Chiang Rai", 11),
    "แม่ฮ่องสอน": ("Mae Hong Son", 23),
    "น่าน": ("Nan", 32),
    "ลำพูน": ("Lamphun", 20),
    "แพร่": ("Phrae", 47),
    "พะเยา": ("Phayao", 41),
    "ตาก": ("Tak", 69),
    "พิษณุโลก": ("Phitsanulok", 45),
    "อุตรดิตถ์": ("Uttaradit", 75),
    "กำแพงเพชร": ("Kamphaeng Phet", 15),
    "พิจิตร": ("Phichit", 44),
    "สุโขทัย": ("Sukhothai", 65),
    "เพชรบูรณ์": ("Phetchabun", 42),
    "อุทัยธานี": ("Uthai Thani", 74),
    "สุราษฏร์ธานี": ("Surat Thani", 67),
    "ภูเก็ต": ("Phuket", 48),
    "สงขลา": ("Songkhla", 64),
    "นราธิวาส": ("Narathiwat", 33),
    "ยะลา": ("Yala", 76),
    "สตูล": ("Satun", 61),
    "นครศรีธรรมราช": ("Nakhon Si Thammarat", 31),
    "ตรัง": ("Trang", 70),
    "สระบุรี": ("Saraburi", 60),
    "สระแก้ว": ("Sa Kaeo", 55)
}

df_info['จังหวัด'] = df_info['จังหวัด'].str.replace(r'^จ\.\s*', '', regex=True)
df_info["english_name"] = df_info["จังหวัด"].map(lambda x: thai_to_eng_map.get(x, ("", ""))[0])

print(df_info)
print("--"*20)
print(len(thai_to_eng_map))

    รหัสสถานี                                 ชื่อสถานี  \
0         54T                 แขวงดินแดง เขตดินแดง กทม.   
1         30T               ต.ท่าประดู่ อ.เมือง จ.ระยอง   
2         36T             ต.ศรีภูมิ อ.เมือง จ.เชียงใหม่   
4         24T  ต.หน้าพระลาน อ.เฉลิมพระเกียรติ จ.สระบุรี   
7         44T               ต.หาดใหญ่ อ.หาดใหญ่ จ.สงขลา   
..        ...                                       ...   
479      109T                 ต.ในเมือง อ.เมือง จ.ยโสธร   
480      110T               ต.บุ่ง อ.เมือง จ.อำนาจเจริญ   
481      111T             ต.นอกเมือง อ.เมือง จ.สุรินทร์   
482      112T              ต.หนองครก อ.เมือง จ.ศรีสะเกษ   
496      105T              ต.บ้านใหญ่ อ.เมือง จ.นครนายก   

         รายละเอียดจุดติดตั้งสถานี     จังหวัด        english_name  
0               การเคหะชุมชนดินแดง        กทม.  Bangkok Metropolis  
1        สำนักงานเกษตรจังหวัดระยอง       ระยอง              Rayong  
2           โรงเรียนยุพราชวิทยาลัย   เชียงใหม่          Chiang Mai  
4     สถานีตำรว

In [46]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, 0 to 496
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   รหัสสถานี                  101 non-null    object
 1   ชื่อสถานี                  101 non-null    object
 2   รายละเอียดจุดติดตั้งสถานี  101 non-null    object
 3   จังหวัด                    101 non-null    object
 4   english_name               101 non-null    object
dtypes: object(5)
memory usage: 4.7+ KB


In [47]:
df_info.head()

,รหัสสถานี,ชื่อสถานี,รายละเอียดจุดติดตั้งสถานี,จังหวัด,english_name
0,54T,แขวงดินแดง เขตดินแดง กทม.,การเคหะชุมชนดินแดง,กทม.,Bangkok Metropolis
1,30T,ต.ท่าประดู่ อ.เมือง จ.ระยอง,สำนักงานเกษตรจังหวัดระยอง,ระยอง,Rayong
2,36T,ต.ศรีภูมิ อ.เมือง จ.เชียงใหม่,โรงเรียนยุพราชวิทยาลัย,เชียงใหม่,Chiang Mai
4,24T,ต.หน้าพระลาน อ.เฉลิมพระเกียรติ จ.สระบุรี,สถานีตำรวจภูธรตำบลหน้าพระลาน,สระบุรี,Saraburi
7,44T,ต.หาดใหญ่ อ.หาดใหญ่ จ.สงขลา,เทศบาลนครหาดใหญ่,สงขลา,Songkhla


In [48]:
df_info = df_info[df_info["english_name"].isin(["Bangkok Metropolis", "Phuket", "Chon Buri", "Kanchanaburi", "Chiang Mai", "Khon Kaen"])].reset_index(drop=True)

check if we have all province in data (because pm_map do not have all province)

In [49]:
df_info["english_name"].unique()

array(['Bangkok Metropolis', 'Chiang Mai', 'Khon Kaen', 'Chon Buri',
       'Kanchanaburi', 'Phuket'], dtype=object)

### this one really is filtration bruh

In [50]:
station_cols = df_pm.columns[df_pm.columns.str.split('|').str[0].str.strip().isin(df_info["รหัสสถานี"])]
df_new = df_pm.loc[:, ['Date'] + station_cols.tolist()]

In [51]:
df_new

,Date,54T | แขวงดินแดง เขตดินแดง กทม.,36T | ต.ศรีภูมิ อ.เมือง จ.เชียงใหม่,61T | แขวงพลับพลา เขตวังทองหลาง กทม.,46T | ต.ในเมือง อ.เมือง จ.ขอนแก่น,59T | แขวงพญาไท เขตพญาไท กทม.,32T | ต.ทุ่งสุขลา อ.ศรีราชา จ.ชลบุรี,05T | แขวงบางนา เขตบางนา กทม.,52T | แขวงบางยี่เรือ เขตธนบุรี กทม.,35T | ต.ช้างเผือก อ.เมือง จ.เชียงใหม่,...,53T | แขวงวังทองหลาง เขตวังทองหลาง กทม.,79T | ต.ปากแพรก อ.เมือง จ.กาญจนบุรี,10T | แขวงคลองจั่น เขตบางกะปิ กทม.,11T | แขวงดินแดง เขตดินแดง กทม.,03T | แขวงแสมดำ เขตบางขุนเทียน กทม.,33T | ต.บ่อวิน อ.ศรีราชา จ.ชลบุรี,34T | ต.บ้านสวน อ.เมือง จ.ชลบุรี,02T | แขวงหิรัญรูจี เขตธนบุรี กทม.,12T | แขวงช่องนนทรี เขตยานนาวา กทม.,43T | ต.ตลาดใหญ่ อ.เมือง จ.ภูเก็ต
0,2024-12-31 00:00:00,34.9,21.5,29.5,30.0,21.9,20.7,33.9,36.9,26.6,...,34.1,34.4,32.9,30.0,32.5,23.9,22.3,37.1,30.1,15.6
1,2024-12-30 00:00:00,33.4,23.1,27.1,35.2,26.2,19.6,28.4,39.2,27.3,...,32.7,24.7,28.2,28.4,34.3,23.4,23.0,40.0,28.1,13.7
2,2024-12-29 00:00:00,23.3,20.1,20.1,NaN,17.0,16.8,20.1,25.2,24.2,...,23.6,24.4,20.4,20.6,22.3,17.5,15.4,25.8,20.7,10.1
3,2024-12-28 00:00:00,27.7,17.4,22.6,24.5,17.0,15.7,22.9,27.8,22.8,...,27.9,28.6,22.5,22.8,25.6,19.0,17.8,28.2,23.4,9.8
4,2024-12-27 00:00:00,44.5,23.6,34.6,24.5,23.2,27.6,35.4,46.2,27.3,...,40.0,NaN,35.8,35.4,43.9,25.9,20.7,47.4,35.9,12.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4974,2011-05-20 00:00:00,43,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4975,2011-05-19 00:00:00,49,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4976,2011-05-18 00:00:00,45,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4977,2011-05-17 00:00:00,45,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## ANDDDDD SAVEE FINISHED

In [52]:
df_new.to_csv("../../data/a1/pm_map.csv", index=False)